#  Matching Catalogs by Members ID

---
**License**

 Matching Catalogs by Members ID

 Tue Aug 24 10:30:00 2021\
 Copyright  2021\
 Cinthia Nunes de Lima <cinthia.n.lima@uel.br> \ Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 Ascaso_Extended_cDC2_RM_Test\
 Copyright (C) 2021 Cinthia Nunes de Lima <cinthia.n.lima@uel.br>, Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

## Packages

In [19]:
import math
import sys

import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table, join, unique
from astropy.coordinates import SkyCoord
import astropy.units as u

import pandas as pd

sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
sys.path.insert(0, "/global/homes/c/cinlima/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/")


import GCRCatalogs
#Put nersc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)


from numcosmo_py import Nc, Ncm

from match_id import MatchID as MID

import time

# GCRCatalogs.get_available_catalog_names()
print('Numpy =', np.__version__)

GCRCatalogs = 1.9.0 | GCR = 0.9.2
Numpy = 1.26.4


## Catalogs

In [151]:
# #Cosmo DC2
# cosmodc2= GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

# #Cosmo DC2 + RedMaPPer
# catalog_dc2_redmapper = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

# catalog_dc2_redmapper.get_catalog_info()

# Restrictions on richness, mass and redshift
# min_richness = 0
# min_halo_mass = 1e13 #Msun
# redshift_max = 1.2

# #cDC2 + RM data
# quantities = catalog_dc2_redmapper.list_all_quantities()
# cluster_quantities = [q for q in quantities if 'member' not in q]
# member_quantities = [q for q in quantities if 'member' in q]
    
# # query = GCRCatalogs.GCRQuery('(richness > ' + str(min_richness) +')')
# cluster_data = Table(catalog_dc2_redmapper.get_quantities(cluster_quantities)) # cDC2 + RM cluster data  [query])
# member_data = Table(catalog_dc2_redmapper.get_quantities(member_quantities)) # cDC2 + RM member data

# #cDC2 data
# quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']
# quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central', 'stellar_mass']
# query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +') & (is_central < ' + str(redshift_max) +') ')
# t_data = Table(cosmodc2.get_quantities(quantities_wanted, [query]))

# cluster_data = Table.read('cluster_data.fits')
# member_data = Table.read('member_data.fits')
# truth_data = Table.read('truth_data.fits')


In [152]:
cluster_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/cluster_data.fits')
member_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/member_data.fits')
truth_data = Table.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching/truth_data.fits')

cluster_member_data = join(member_data, cluster_data, keys_left = 'cluster_id_member', keys_right = 'cluster_id', join_type='inner')

# memberdf = member_data.to_pandas().rename(columns={'id_member': 'id', 'cluster_id_member': 'cluster_id'}) #This creates a dataframe and rename: id_member -> id (cDC2 + RM MEMBERS).
# truthdf = truth_data.to_pandas().rename(columns={'galaxy_id': 'id'}) #This creates a dataframe and rename: galaxy_id -> id (cDC2 MEMBERS).
# cluster_df=cluster_data.to_pandas()[['richness', 'richness_err', 'cluster_id', 'cluster_redshift', 'cluster_redshift_err', 'cluster_ra', 'cluster_dec']].rename(columns={'ra': 'cluster_ra', 'dec': 'cluster_dec', 'redshift': 'cluster_redshift', 'redshift_err': 'cluster_redshift_err'}) #This creates a dataframe for cDC2 clusters.

truthdf = truth_data.to_pandas()
hascentral = truthdf[truthdf['is_central'] ==  True]

halo_central_ids = list(hascentral['halo_id'].unique())

truth_data = Table.from_pandas(truthdf[truthdf['halo_id'].isin(halo_central_ids)]) #just halos with central galaxies

# Match

### functions

In [153]:
def get_members_number(catalog, object_tag):

   members_number = catalog[object_tag].value_counts()

   return members_number, pd.merge(pd.DataFrame(members_number), catalog, how='inner', on=[object_tag])     

def get_shared_members_fraction_catalog(matched_catalog):
   
    shared_count = matched_catalog.groupby(['cat1_id', 'cat2_id'])['cat2_id'].transform('count') 
    
    frac_catalog1 = shared_count / matched_catalog['nmem_cat1']
    frac_catalog2 = shared_count / matched_catalog['nmem_cat2']

    return shared_count, frac_catalog1, frac_catalog2

def get_angular_distance(ra_1, dec_1, ra_2, dec_2):

    ra_1 = np.asarray(ra_1, dtype=float)
    dec_1 = np.asarray(dec_1, dtype=float)
    ra_2 = np.asarray(ra_2, dtype=float)
    dec_2 = np.asarray(dec_2, dtype=float)
    
    coords_1 = SkyCoord(ra=ra_1 * u.deg,
                          dec=dec_1 * u.deg,
                          frame='icrs')

    coords_2 = SkyCoord(ra=ra_2 * u.deg,
                           dec=dec_2 * u.deg,
                           frame='icrs')

    return coords_1.separation(coords_2).degree

def load_catalogs(catalog1, catalog2, id_columns1, id_columns2 ):

   cat1 = catalog1.to_pandas().rename(columns = id_columns1)    
   cat2 = catalog2.to_pandas().rename(columns = id_columns2)

   #Number of members associated with the objects (e.g. clusters or halos):  
   cat1_nmem = get_members_number(cat1, 'cat1_id')[1].rename(columns={'count': 'nmem_cat1'})
   cat2_nmem = get_members_number(cat2, 'cat2_id')[1].rename(columns={'count': 'nmem_cat2'})
   
   return cat1_nmem, cat2_nmem

def match_by_id(catalog1, catalog2):

    #Matching
    matched_catalog = pd.merge(catalog1, catalog2, how='inner', on=['id'])   
    
    # Shared members fraction
    shared_frac = get_shared_members_fraction_catalog(matched_catalog)

    matched_catalog['shared_num'] = shared_frac[0]
    matched_catalog['shared_frac_cat1'] = shared_frac[1]
    matched_catalog['shared_frac_cat2'] = shared_frac[2]

    #Angular distance
    ang_distance = get_angular_distance(matched_catalog['cat1_ra'], matched_catalog['cat1_dec'], 
                                        matched_catalog['cat2_ra'], matched_catalog['cat2_dec'])
    
    matched_catalog['ang_distance'] = ang_distance
    
    
    return matched_catalog

def get_max_fraction_candidates(object_data, canditate_object_tag, shared_fraction_option = 'shared_frac_cat1'):
   
    max_fraction = object_data[shared_fraction_option].max()
    
    return object_data[object_data[shared_fraction_option] == max_fraction][canditate_object_tag].unique()


In [154]:
cat1_columns = {'id_member': 'id', 'cluster_id': 'cat1_id', 'cluster_ra': 'cat1_ra', 'cluster_dec': 'cat1_dec', 'cluster_redshift':'cat1_redshift'}
cat2_columns = {'galaxy_id': 'id', 'halo_id': 'cat2_id', 'ra': 'cat2_ra', 'dec': 'cat2_dec', 'redshift':'cat2_redshift'}

catalogs = load_catalogs(cluster_member_data, truth_data, cat1_columns, cat2_columns)

cat_clusters = catalogs[0]
cat_halos = catalogs[1]

matched_catalog = match_by_id(cat_clusters, cat_halos)

In [155]:
matched_catalog = matched_catalog.rename(columns={'cat1_id':'cluster_id' , 'cat1_ra': 'cluster_ra' ,
                                'cat1_dec': 'cluster_dec' , 'cat1_redshift': 'cluster_redshift',
                                'cat2_id': 'halo_id' , 'cat2_ra':'ra' , 'cat2_dec': 'dec' ,
                                'cat2_redshift': 'redshift', 'shared_frac_cat1': 'shared_frac_cluster',
                                                 'shared_frac_cat2': 'shared_frac_halo'})
matched_catalog

,cluster_id,nmem_cat1,mag_y_lsst_member,redshift_true_member,mag_i_lsst_member,magerr_y_lsst_member,dec_member,mag_z_lsst_member,p_member,pfree_member,...,nmem_cat2,ra,redshift,dec,is_central,halo_mass,shared_num,shared_frac_cluster,shared_frac_halo,ang_distance
0,8,333,19.499664,0.363710,20.042974,0.015663,-37.495165,19.735102,0.954799,1.0,...,289,60.815059,0.363710,-37.495165,False,5.070324e+14,49,0.147147,0.169550,0.067176
1,8,333,20.022087,0.365097,20.537331,0.024289,-37.515030,20.201677,0.553526,1.0,...,278,60.660835,0.365097,-37.515030,False,2.508649e+14,25,0.075075,0.089928,0.058470
2,8,333,19.627020,0.341498,20.160690,0.018432,-37.487604,19.846445,0.945550,1.0,...,714,60.657031,0.341498,-37.487604,False,5.607389e+14,141,0.423423,0.197479,0.058769
3,8,333,17.726294,0.328147,18.200514,0.012003,-37.484271,17.908180,0.954372,1.0,...,714,60.803638,0.328147,-37.484271,False,5.607389e+14,141,0.423423,0.197479,0.059229
4,8,333,20.161432,0.366350,20.711756,0.017306,-37.445372,20.391184,0.953338,1.0,...,278,60.690734,0.366350,-37.445372,False,2.508649e+14,25,0.075075,0.089928,0.059356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542710,25034,6,19.083742,0.481923,19.667698,0.015089,-37.532624,19.288912,0.958204,1.0,...,21,62.349620,0.481923,-37.532624,False,1.286829e+13,3,0.500000,0.142857,0.000000
542711,22626,6,18.458124,0.442705,19.007614,0.015683,-30.179373,18.664827,0.453264,1.0,...,11,64.807554,0.442705,-30.179373,False,1.416681e+13,4,0.666667,0.363636,0.000000
542712,22626,6,19.966616,0.444697,20.610235,0.026292,-30.184543,20.191570,0.913804,1.0,...,11,64.800115,0.444697,-30.184543,False,1.416681e+13,4,0.666667,0.363636,0.008251
542713,22626,6,18.866642,0.443212,19.415993,0.012822,-30.179809,19.052666,0.957062,1.0,...,11,64.810874,0.443212,-30.179809,True,1.416681e+13,4,0.666667,0.363636,0.002903


### match

In [156]:
inicio = time.time()

clusters_id = matched_catalog['cluster_id'].unique() 
cluster_groups = matched_catalog.groupby('cluster_id') 


match_info_cluster = pd.DataFrame(dtype = 'Int64')

for cl in clusters_id:

    cl_group = cluster_groups.get_group(cl).copy()
    matched_halos_id = list(cl_group['halo_id'].unique())
    
    
    ## BEST CANDIDATE SELECTION

    best_candidate = []

    
    # 1: Max shared fraction
    max_frac_cluster_candidates = cl_group[cl_group['shared_frac_cluster'] == cl_group['shared_frac_cluster'].max()]['halo_id'].unique()

    
    # 2: Minimum distance between cluster and halo (if it has more than 2 candidates)
    if len(max_frac_cluster_candidates) != 1:

        candidates_info = cl_group[cl_group['halo_id'].isin(max_frac_cluster_candidates)]

        ch_distances = []
        
        for mfc in max_frac_cluster_candidates:

            candidate = candidates_info[candidates_info['halo_id'] == mfc]
            
            if True in candidate['is_central'].tolist():

                candidate_central = candidate[candidate['is_central'] == True]

                cluster_halo_distance = get_angular_distance(candidate_central['cluster_ra'].values, candidate_central['cluster_dec'].values,
                                                               candidate_central['ra'].values, candidate_central['dec'].values)
                
            else:

                # candidate_ra = np.mean(candidate['ra'])
                # candidate_dec = np.mean(candidate['dec'])
                
                # cluster_halo_distance = get_angular_distance(candidate['cluster_ra'].values, candidate['cluster_dec'].values,
                #                                                candidate_ra, candidate_dec)
                pass
                
            
            ch_distances.append((mfc, cl, cluster_halo_distance[0]))

        ch_distances_df = pd.DataFrame(ch_distances, columns= ['halo_id', 'cluster_id', 'distance'])
        min_distance_candidates = ch_distances_df[ch_distances_df['distance'] == ch_distances_df['distance'].min()]
       
        best_candidate.append(min_distance_candidates['halo_id'].iloc[0])

    else:
        
        best_candidate.append(max_frac_cluster_candidates[0])
           
    
    # MATCHING INFORMATION
   
    for hl in matched_halos_id:

        h_group = cl_group.groupby('halo_id').get_group(hl)
        
        matched_members_id = h_group['id']
        shared_num = h_group['shared_num'].iloc[0]
        shared_frac_cluster = h_group['shared_frac_cluster'].iloc[0]

        if hl == best_candidate[0]:
            best = True
            
        else:
            best = False
                       
        info = pd.DataFrame([{'cluster_id': cl, 'halo_id': hl, 'is_best': best, 'shared_num': shared_num, 
                                               'shared_frac_cluster': shared_frac_cluster,
                                               'matched_members_id':matched_members_id.tolist(), 
                                               'matched_halos_id': matched_halos_id}])
               
    
        match_info_cluster = pd.concat([match_info_cluster, info], ignore_index=True)



fim = time.time()
tempo_total = fim - inicio

print(f"Tempo de execução: {tempo_total/60:.3}min")

Tempo de execução: 2.22min


In [157]:
match_info_cluster

,cluster_id,halo_id,is_best,shared_num,shared_frac_cluster,matched_members_id,matched_halos_id
0,8,90000127355,False,49,0.147147,"[7937689640, 7937689531, 7937698893, 793768973...","[90000127355, 595400141355, 536900141373, 2927..."
1,8,595400141355,False,25,0.075075,"[8812691075, 8812690980, 8812691013, 881269099...","[90000127355, 595400141355, 536900141373, 2927..."
2,8,536900141373,True,141,0.423423,"[8812617877, 8812617833, 8812617822, 881261734...","[90000127355, 595400141355, 536900141373, 2927..."
3,8,292700141373,False,74,0.222222,"[8812618517, 8812613821, 8812618535, 881261853...","[90000127355, 595400141355, 536900141373, 2927..."
4,8,23400127355,False,2,0.006006,"[7937688607, 7937688598]","[90000127355, 595400141355, 536900141373, 2927..."
...,...,...,...,...,...,...,...
80912,29448,280500066315,True,4,0.666667,"[4125528281, 4125528294, 4125528276, 4125528286]",[280500066315]
80913,29458,16400153432,True,5,0.833333,"[9562509660, 9562509659, 9562509667, 956250966...",[16400153432]
80914,24154,501400107323,True,6,1.000000,"[6687943745, 6687943743, 6687943744, 668794373...",[501400107323]
80915,25034,28700127331,True,3,0.500000,"[7937868008, 7937868001, 7937868013]",[28700127331]


In [158]:
best = match_info_cluster[match_info_cluster['is_best']== True]
best

,cluster_id,halo_id,is_best,shared_num,shared_frac_cluster,matched_members_id,matched_halos_id
2,8,536900141373,True,141,0.423423,"[8812617877, 8812617833, 8812617822, 881261734...","[90000127355, 595400141355, 536900141373, 2927..."
8,1,2050900064279,True,275,0.881410,"[4001174186, 4001174419, 4001174188, 400117424...","[2050900064279, 2608400064279]"
11,3,101900137401,True,103,0.334416,"[8562545950, 8562545963, 8562541999, 856254211...","[9600137401, 101900137401, 173200123401, 19240..."
17,4,269600069315,True,277,0.914191,"[4313033142, 4313033011, 4313053214, 431303364...","[269600069315, 1951800069307, 686000069307]"
20,6,92300053338,True,251,0.929630,"[3312792427, 3312792000, 3312806283, 331280625...",[92300053338]
...,...,...,...,...,...,...,...
80912,29448,280500066315,True,4,0.666667,"[4125528281, 4125528294, 4125528276, 4125528286]",[280500066315]
80913,29458,16400153432,True,5,0.833333,"[9562509660, 9562509659, 9562509667, 956250966...",[16400153432]
80914,24154,501400107323,True,6,1.000000,"[6687943745, 6687943743, 6687943744, 668794373...",[501400107323]
80915,25034,28700127331,True,3,0.500000,"[7937868008, 7937868001, 7937868013]",[28700127331]


In [159]:
best.sort_values(by= 'halo_id')

,cluster_id,halo_id,is_best,shared_num,shared_frac_cluster,matched_members_id,matched_halos_id
14487,7185,78259,True,23,0.741935,"[4876723482, 4876723406, 4876723346, 487672344...","[78259, 3584500078259, 1474400078259]"
77483,29091,80442,True,9,0.900000,"[5000004454, 5000004464, 5000004488, 500000448...",[80442]
22758,17063,100094347,True,4,0.166667,"[5875240286, 5875240276, 5875240283, 5875240280]",[100094347]
33479,20166,100163315,True,8,0.400000,"[10188014163, 10188014166, 10188014150, 101880...","[573100163307, 100163315, 1464500163307]"
36233,15005,100172347,True,8,0.421053,"[10750235967, 10750235973, 10750235969, 107502...","[100172347, 521000172355]"
...,...,...,...,...,...,...,...
52055,42196,4995900144224,True,3,0.200000,"[9003399374, 9003399358, 9003399359]",[4995900144224]
63650,54174,5009300144224,True,5,0.384615,"[9003399544, 9003399569, 9003399564, 900339957...","[5009300144224, 491000144247]"
27402,19561,5089100155224,True,11,0.500000,"[9690902852, 9690902872, 9690902799, 969090286...","[5089100155224, 3804000155224]"
32388,63569,5111100158224,True,7,0.350000,"[9878399356, 9878399383, 9878399398, 987839936...","[5111100158224, 1666600158224, 5053300158224, ..."


In [160]:
sorted_multiple = best.sort_values(by=['halo_id', 'shared_frac_cluster'], ascending=[True, False])

# best.drop_duplicates(subset=['halo_id'], keep='first')

In [30]:
inicio = time.time()

halos_id = matched_catalog['halo_id'].unique() 
halo_groups = matched_catalog.groupby('halo_id') 


match_info_halo = pd.DataFrame(dtype = 'Int64')
for hl in halos_id:

    hl_group = halo_groups.get_group(hl).copy()
    matched_clusters_id = list(hl_group['cluster_id'].unique())
    
    
    ## BEST CANDIDATE SELECTION

    best_candidate = []

    
    # 1: Max shared fraction
    max_frac_halo_candidates = hl_group[hl_group['shared_frac_halo'] == hl_group['shared_frac_halo'].max()]['cluster_id'].unique()

    # print(max_frac_halo_candidates)
    
    # 2: Minimum distance between cluster and halo (if it has more than 2 candidates)
    if len(max_frac_halo_candidates) != 1:

        candidates_info = hl_group[hl_group['cluster_id'].isin(max_frac_halo_candidates)]

        # max_frac_halos_candidates = candidates_info[candidates_info['shared_frac_halo'] == candidates_info['shared_frac_halo'].max()]['halo_id'].unique()

        # print(max_frac_halos_candidates)

        ch_distances = []
        
        for mfc in max_frac_halo_candidates:

            candidate = candidates_info[candidates_info['cluster_id'] == mfc]

            # print(mfc)
            
            if True in candidate['is_central'].tolist():

                candidate_central = candidate[candidate['is_central'] == True]

                cluster_halo_distance = get_angular_distance(candidate_central['cluster_ra'].values, candidate_central['cluster_dec'].values,
                                                               candidate_central['ra'].values, candidate_central['dec'].values)
                
            else:

                candidate_ra = np.mean(candidate['ra'])
                candidate_dec = np.mean(candidate['dec'])
                
                cluster_halo_distance = get_angular_distance(candidate['cluster_ra'].values, candidate['cluster_dec'].values,
                                                               candidate_ra, candidate_dec)

                # pass

            ch_distances.append((mfc, cl, cluster_halo_distance[0]))

        
        ch_distances_df = pd.DataFrame(ch_distances, columns= ['halo_id', 'cluster_id', 'distance'])
        min_distance_candidates = ch_distances_df[ch_distances_df['distance'] == ch_distances_df['distance'].min()]

        best_candidate.append(min_distance_candidates['halo_id'].iloc[0])

    else:
        
        best_candidate.append(max_frac_halo_candidates[0])
           
    
    # MATCHING INFORMATION
   
    for cl in matched_clusters_id:

        c_group = hl_group.groupby('cluster_id').get_group(cl)
        
        matched_members_id = c_group['id']
        shared_num = c_group['shared_num'].iloc[0]
        shared_frac_cluster = c_group['shared_frac_cluster'].iloc[0]

        if cl == best_candidate[0]:
            best = True
            
        else:
            best = False
                       
        info = pd.DataFrame([{'cluster_id': cl, 'halo_id': hl, 'is_best': best, 'shared_num': shared_num, 
                                               'shared_frac_cluster': shared_frac_cluster,
                                               'matched_members_id':matched_members_id.tolist(), 
                                               'matched_cluster_id': matched_clusters_id}])
               
    
        match_info_halo = pd.concat([match_info_halo, info], ignore_index=True)



fim = time.time()
tempo_total = fim - inicio

print(f"Tempo de execução: {tempo_total/60:.3}min")

Tempo de execução: 2.55min


In [31]:
match_info_halo


,cluster_id,halo_id,is_best,shared_num,shared_frac_cluster,matched_members_id,matched_cluster_id
0,8,90000127355,False,49,0.147147,"[7937689640, 7937689531, 7937698893, 793768973...","[8, 37, 29759]"
1,37,90000127355,True,97,0.757812,"[7937689516, 7937698877, 7937698884, 793769887...","[8, 37, 29759]"
2,29759,90000127355,False,1,0.090909,[7937689647],"[8, 37, 29759]"
3,8,595400141355,False,25,0.075075,"[8812691075, 8812690980, 8812691013, 881269099...","[8, 106]"
4,106,595400141355,True,33,0.687500,"[8812691085, 8812690951, 8812691070, 881269096...","[8, 106]"
...,...,...,...,...,...,...,...
80912,29448,280500066315,True,4,0.666667,"[4125528281, 4125528294, 4125528276, 4125528286]",[29448]
80913,29458,16400153432,True,5,0.833333,"[9562509660, 9562509659, 9562509667, 956250966...",[29458]
80914,24154,501400107323,True,6,1.000000,"[6687943745, 6687943743, 6687943744, 668794373...",[24154]
80915,25034,28700127331,True,3,0.500000,"[7937868008, 7937868001, 7937868013]",[25034]


In [65]:
best = match_info_halo[match_info_halo['is_best']== True ]

42829

In [33]:
central = truthdf[truthdf['is_central'] ==  True]

halo_central_ids = list(central['halo_id'].unique())

best = match_info_cluster[match_info_cluster['is_best']== True ]

best[best['halo_id'].isin(halo_central_ids)]

,cluster_id,halo_id,is_best,shared_num,shared_frac_cluster,matched_members_id,matched_halos_id
2,8,536900141373,True,141,0.423423,"[8812617877, 8812617833, 8812617822, 881261734...","[90000127355, 595400141355, 536900141373, 2927..."
8,1,2050900064279,True,275,0.881410,"[4001174186, 4001174419, 4001174188, 400117424...","[2050900064279, 2608400064279]"
11,3,101900137401,True,103,0.334416,"[8562545950, 8562545963, 8562541999, 856254211...","[9600137401, 101900137401, 173200123401, 19240..."
17,4,269600069315,True,277,0.914191,"[4313033142, 4313033011, 4313053214, 431303364...","[269600069315, 1951800069307, 686000069307]"
20,6,92300053338,True,251,0.929630,"[3312792427, 3312792000, 3312806283, 331280625...",[92300053338]
...,...,...,...,...,...,...,...
80912,29448,280500066315,True,4,0.666667,"[4125528281, 4125528294, 4125528276, 4125528286]",[280500066315]
80913,29458,16400153432,True,5,0.833333,"[9562509660, 9562509659, 9562509667, 956250966...",[16400153432]
80914,24154,501400107323,True,6,1.000000,"[6687943745, 6687943743, 6687943744, 668794373...",[501400107323]
80915,25034,28700127331,True,3,0.500000,"[7937868008, 7937868001, 7937868013]",[28700127331]


In [ ]:
best

In [ ]:
match_info

In [ ]:
match_info[match_info['is_best'] == True]

# if True in match_info[match_info['is_best'] == True]['halo_id'].duplicated():
#     print('duplicates')
# else:
#     print( "Duplicates don't exists.")


In [ ]:
best_candidates = match_info[match_info['is_best'] == True]

halo_g = truthdf.groupby('halo_id')

match_dataframe = pd.DataFrame(data = {'cluster_id':[], 'halo_id': [], 'halo_redshift': [], 'cluster_ra': [],
                                                   'cluster_dec': [], 'richness': [],
                                                   'richness_err': [], 'halo_mass': [],
                                                   'redshift': [], 'cluster_redshift': [],
                                                   'cluster_redshift_err': []}, dtype = 'Int64')


for cl, ha in zip(best_candidates['cluster_id'], best_candidates['halo_id']):

    cluster_data = matched_catalog[matched_catalog['cluster_id'] == cl]

    cluster_ra, cluster_dec, richness, richness_err, cluster_redshift, cluster_redshift_err = cluster_data['cluster_ra'].iloc[0], cluster_data['cluster_dec'].iloc[0], cluster_data['richness'].iloc[0], cluster_data['richness_err'].iloc[0], cluster_data['cluster_redshift'].iloc[0], cluster_data['cluster_redshift_err'].iloc[0]

    halo_mass =  halo_g.get_group(ha)['halo_mass'].iloc[0]

    halo_redshift = np.mean(halo_g.get_group(ha)[['redshift']])


    all_data = pd.DataFrame([{'cluster_id':cl, 'halo_id': ha, 'halo_redshift': halo_redshift,'cluster_ra': cluster_ra, 'cluster_dec': cluster_dec,
                                                       'richness': richness, 'richness_err': richness_err, 'halo_mass': halo_mass,
                                                       'redshift': halo_redshift, 'cluster_redshift': cluster_redshift,
                                                       'cluster_redshift_err': cluster_redshift_err}])

    
                         
    match_dataframe = pd.concat([match_dataframe, all_data], ignore_index=True)




In [ ]:
match_dataframe

In [ ]:
Table.from_pandas(match_dataframe).write('match_ID.fits', overwrite=True)

## Data Analysis

### Duplicates

In [ ]:
if True in match_dataframe_organized['halo_id'].duplicated():
    match_dataframe_organized[match_dataframe_organized['halo_id'].duplicated()]
else:
    print( "Duplicated halos don't exists.")

In [ ]:
if True in match_dataframe_organized['cluster_id'].duplicated():
    match_dataframe_organized[match_dataframe_organized['cluster_id'].duplicated()]
else:
    print( "Duplicated clusters don't exists.")

In [ ]:
# if True in match_dataframe_organized['id'].duplicated():
#     match_dataframe_organized[match_dataframe_organized['id'].duplicated()]
# else:
#     print( "Duplicated members don't exists.")

### RA, Dec, Redshift

In [ ]:
# plt.scatter(match_dataframe_organized['cluster_ra'], match_dataframe_organized['halo_ra'])

# plt.xlabel('Cluster RA')
# plt.ylabel('Halo RA')
# plt.show()

In [ ]:
# plt.scatter(match_dataframe_organized['cluster_dec'], match_dataframe_organized['halo_dec'])

# plt.xlabel('Cluster Dec')
# plt.ylabel('Halo Dec')
# plt.show()

In [ ]:
# zdf = match_dataframe_organized.sort_values(by='redshift')

# x = match_dataframe_organized['redshift']
# y = match_dataframe_organized['cluster_redshift']
# y_err = match_dataframe_organized['cluster_redshift_err']

# # plt.scatter(x, y)

# plt.errorbar(x, y, yerr = y_err, fmt=".")

# plt.xlabel('Cluster redshift')
# plt.ylabel('Halo redshift')
# plt.show()



### Richness x Mass

In [ ]:
plt.scatter(np.log(match_dataframe['halo_mass']), np.log(match_dataframe['richness']))

plt.xlabel('log10 M ')
plt.ylabel('ln R')
plt.show()

## Notes

### Notes 1

In [ ]:
# member_data.write('member_data.fits')

In [ ]:
# truth_data[truth_data['is_central'] == True]

# cluster_data.to_pandas().head()

# type(member_data)

# cluster_data.rename_column('ra', 'cluster_ra')
# cluster_data.rename_column('dec', 'cluster_dec')
# cluster_data.rename_column('redshift', 'cluster_redshift')
# cluster_data.rename_column('redshift_err', 'cluster_redshift_err')

In [ ]:
# inicio = time.time()


# mid = MID(member_data, cluster_data, truth_data)
# matchdf = mid.unique_id_match(distance=False)
# fim = time.time()
# tempo_total = fim - inicio

# print(f"Tempo de execução: {tempo_total/60:.3}min")

In [ ]:
# print(matchdf['distance_2d'].min(), matchdf['distance_2d'].max())

# matchdf

In [ ]:
# Mt_df = pd.merge(memberdf['id'], truthdf['id'], how='inner', on=['id']) #This selects commom members ID from cDC2 and cDC2 + RM data
# Mt_df

In [ ]:
# mt_member = pd.merge(memberdf, Mt_df, how='inner', on=['id']) #This selects the matching members using Mt_df IDs
# mt_member

In [ ]:
# idc = mt_member[['id', 'cluster_id_member']].rename(columns={'cluster_id_member': 'cluster_id'}) #This selects the members and clusters IDs
# idc

In [ ]:
# mt_cluster = pd.merge(idc, cluster_df, how='inner', on=['cluster_id']) #This selects the clusters data by idc IDS 
# mt_cluster

In [ ]:
# mt_catalog = pd.merge(mt_cluster, truthdf, how='inner', on=['id']) #This creates the matching catalog with clustes and halos
# mt_catalog

### Notes 2

In [ ]:
# Criar o ID que conecta os dois catálogos
shared_ids = np.arange(1001, 1011)  # 10 IDs de conexão

# Criar IDs exclusivos para o catálogo 1 (3 diferentes)
catalog1_unique_ids = np.random.choice([200, 201, 202], size=10)

# Criar IDs exclusivos para o catálogo 2 (4 diferentes)
catalog2_unique_ids = np.random.choice([300, 301, 302, 303], size=10)

# Criar catálogo 1
catalog1 = pd.DataFrame({
    'shared_id': shared_ids,
    'cat1_unique_id': catalog1_unique_ids,
    'prop1': np.random.randint(1, 100, size=10),
    'prop2': np.random.random(size=10).round(2),
    'prop3': np.random.choice(['A', 'B', 'C'], size=10)
})

# Criar catálogo 2
catalog2 = pd.DataFrame({
    'shared_id': shared_ids,
    'cat2_unique_id': catalog2_unique_ids,
    'prop4': np.random.randint(100, 200, size=10),
    'prop5': np.random.random(size=10).round(2),
    'prop6': np.random.choice(['X', 'Y', 'Z'], size=10)
})



In [ ]:
catalog1

In [ ]:
catalog1['cat1_unique_id'].value_counts()

In [ ]:
catalog2

In [ ]:
match_catalogs = pd.merge(catalog1, catalog2, how='inner', on=['shared_id'])
match_catalogs

In [ ]:
shared_catalog = match_catalogs.groupby(['cat1_unique_id', 'cat2_unique_id'])['cat2_unique_id'].transform('count') 
num_mem1 = match_catalogs.groupby(['cat1_unique_id'])['cat1_unique_id'].transform('count') 
num_mem2 = match_catalogs.groupby(['cat2_unique_id'])['cat2_unique_id'].transform('count') 

frac_catalog1 = shared_catalog / num_mem1

match_catalogs['shared_catalog'] = shared_catalog
match_catalogs['num_mem1'] = num_mem1
match_catalogs['num_mem2'] = num_mem2
match_catalogs['frac_catalog1'] = frac_catalog1



In [ ]:
match_catalogs

### Notes 3

In [ ]:
halodf = truthdf[['halo_id', 'halo_mass', 'redshift', 'ra', 'dec', 'is_central']]
halo_central_galaxies = halodf[halodf['is_central'] == True]

if True in halo_central_galaxies['halo_id'].duplicated():
    halo_central_galaxies[halo_central_galaxies['halo_id'].duplicated()]
else:
    print( "Duplicated galaxies don't exists.")

In [ ]:
truthdf[truthdf['halo_id'] == 3919900022224]

In [ ]:
matched_catalog.groupby('cluster_id').get_group(10)['halo_id'].unique()

matched_catalog.groupby('cluster_id').get_group(10)['nmem_cluster'].unique()[0]

clusters_id = matched_catalog['cluster_id'].unique() #This selects the clusters IDs

matched_catalog.groupby(['cluster_id', 'halo_id']).get_group((10, 1162500021323))['nmem_cluster'] #.count()

In [ ]:
matched_catalog.groupby('cluster_id').get_group(10)['shared_mem']

In [ ]:
# shared_frac_cluster = shared_num / cl_group['nmem_cluster'].unique()[0]

In [ ]:
# match_info[match_info['cluster_id'] == 10  ]

cg = matched_catalog.groupby('cluster_id').get_group(10)

cg[cg['shared_frac_cluster'] == cg['shared_frac_cluster'].max()]


In [ ]:
inicio = time.time()
fim = time.time()

print(f'total time = {fim - inicio :.2f} seconds')


In [ ]:
# cluster_groups = match_dataframe_organized.groupby('cluster_id')

# df_test = pd.DataFrame()

# for cl in clusters_id:

#     cluster_group = cluster_groups.get_group(cl)
#     members = list(cluster_group['id'])   
     
#     cluster_group['members'] = cluster_group['coluna'] = [members] * len(cluster_group) 
    
#     pd.concat([df_test, cluster_group], ignore_index=True)


In [ ]:
matched_catalog.groupby('cluster_id').get_group(24)[matched_catalog.groupby('cluster_id').get_group(24)['is_central'] == True]

In [ ]:
# np.mean(matched_catalog.groupby('cluster_id').get_group(24)[['redshift']])

# np.std(matched_catalog.groupby('cluster_id').get_group(10)[['redshift']])

In [ ]:
# clusters_id = matched_catalog['cluster_id'].unique() 

# for cl in clusters_id:

#     z_mean = np.mean(matched_catalog.groupby('cluster_id').get_group(cl)[['redshift']])
#     matched_catalog.groupby('cluster_id').get_group(24)[['redshift', 'cluster_redshift']]



In [ ]:
group = matched_catalog.groupby('cluster_id').get_group(18)
group.groupby('halo_id').get_group(549500032315)['is_central']


In [ ]:
# group[group['shared_frac_cluster'] == group['shared_frac_cluster'].max()]['shared_frac_cluster'].iloc[0]

In [ ]:
# set(best_candidates['halo_id']) - set(bestc['halo_id'])

# set(bestc['halo_id']).symmetric_difference(set(best_candidates['halo_id']))

In [ ]:
# best_candidates[best_candidates['halo_id'] == 632400183300]
best_candidates[best_candidates['cluster_id'] == 27079]

In [ ]:
match_info[match_info['cluster_id'] == 27079]

In [ ]:
cluster_df[cluster_df['cluster_id'] == 10]['cluster_ra'].iloc[0]

halo_central_galaxies[halo_central_galaxies['halo_id'] == 3919900022224]

# best_candidates[best_candidates['halo_id'] == 3919900022224]

In [ ]:
gal11003719599 = truthdf[truthdf['halo_id'] == 3696200176219]
ra = gal11003719599['ra'].values
dec = gal11003719599['dec'].values

In [ ]:
gal11003719599

In [ ]:
cluster1736 = cluster_df[cluster_df['cluster_id'] == 1736][['cluster_id','cluster_ra', 'cluster_dec', ]]

cluster1736['distance'] = angular_distance(cluster1736['cluster_ra'].values, cluster1736['cluster_dec'].values, ra, dec)
cluster1736
# np.sort(list(memberdf[memberdf['cluster_id'] == 1736]['id']))

In [ ]:
# inicio = time.time()

# mt_dataframe2 = mtrmdc2(member_data, cluster_data, truth_data)

# fim = time.time()
# tempo_total = fim - inicio

# print(f"Tempo de execução: {tempo_total/60:.3}min")

In [ ]:
# mt_dataframe

In [ ]:
# mt_table = Table.from_pandas(match_dataframe)

In [ ]:
# mt_table.write('match_ID.fits')

In [ ]:
# match_dataframe_organized = match_dataframe_organized[match_dataframe_organized['cluster_id'] == 23]

In [ ]:
# match_dataframe_organized = match_dataframe_organized[match_dataframe_organized['richness'] > 50]
# m_selection = match_dataframe_organized[np.logical_and(np.log(match_dataframe_organized['richness']) > 5.0,
#                                                        np.log(match_dataframe_organized['richness'] < 5.5))]

# m_selection = m_selection[np.logical_and(m_selection['redshift'] > 0.2, m_selection['redshift'] < 0.4 )]

# m_selection

In [ ]:
# match_dataframe[ match_dataframe['cluster_id'] == 1059]

In [ ]:
# num_mem1 = memberdf['cluster_id'].value_counts()
# cluster_df = pd.merge(pd.DataFrame(num_mem1), cluster_df, how='inner', on=['cluster_id']) 


In [ ]:
num_mem2 = truthdf['halo_id'].value_counts()
truthdf = pd.merge(pd.DataFrame(num_mem2), truthdf, how='inner', on=['halo_id']) 

In [ ]:
truthdf = truthdf.rename(columns={'count': 'nmem_halo'})
cluster_df = cluster_df.rename(columns={'count': 'nmem_cluster'})

In [ ]:
cluster_catalog = pd.merge(memberdf, cluster_df, how='inner', on=['cluster_id'])
# cluster_catalog

In [ ]:
matched_catalog = pd.merge(cluster_catalog, truthdf, how='inner', on=['id'])
matched_catalog

In [ ]:
shared_count = matched_catalog.groupby(['cluster_id', 'halo_id'])['halo_id'].transform('count') 

frac_catalog1 = shared_count / matched_catalog['nmem_cluster']
frac_catalog_halos = shared_count / matched_catalog['nmem_halo']

matched_catalog['shared_mem'] = shared_count
matched_catalog['shared_frac_cluster'] = frac_catalog1
matched_catalog['shared_frac_halo'] = frac_catalog_halos


In [ ]:
matched_catalog[matched_catalog['halo_id'].isin([1544700047300, 1316400047300])]

In [ ]:
    

# def maximum_shared_fraction(cluster_group):
    
#     return cluster_group[cluster_group['shared_frac_cluster'] == cluster_group['shared_frac_cluster'].max()]


# def minimum_distance(cluster_group, max_frac_candidates):

#     candidates_info = cluster_group[cluster_group['halo_id'].isin(max_frac_candidates)]

#     ch_distances = []
        
#     for mfc in max_frac_candidates:

#         candidate = candidates_info[candidates_info['halo_id'] == mfc]
            
#         if True in candidate['is_central'].tolist():

#                candidate_central = candidate[candidate['is_central'] == True]

#                cluster_halo_distance = angular_distance(candidate_central['cluster_ra'].values, candidate_central['cluster_dec'].values,
#                                                                candidate_central['ra'].values, candidate_central['dec'].values)
                
#         else:

#             candidate_ra = np.mean(candidate['ra'])
#             candidate_dec = np.mean(candidate['dec'])
            
#             cluster_halo_distance = angular_distance(candidate['cluster_ra'].values, candidate['cluster_dec'].values,
#                                                                candidate_ra, candidate_dec)
                
            
#         ch_distances.append((mfc, cl, cluster_halo_distance[0]))

        
#     ch_distances_df = pd.DataFrame(ch_distances, columns= ['halo_id', 'cluster_id', 'distance'])
    
#     min_distance_candidates = ch_distances_df[ch_distances_df['distance'] == ch_distances_df['distance'].min()]

#     return min_distance_candidates
    


In [ ]:
diff = np.isin(best['cluster_id'], clevar_clusters, invert = True)

best[diff]

In [ ]:
halodf = truthdf[['halo_id', 'halo_mass', 'redshift', 'ra', 'dec', 'is_central']]
halo_central_galaxies = halodf[halodf['is_central'] == True]

if True in halo_central_galaxies['halo_id'].duplicated():
    halo_central_galaxies[halo_central_galaxies['halo_id'].duplicated()]
else:
    print( "Duplicated galaxies don't exists.")

In [ ]:
truthdf[truthdf['halo_id'] == 3919900022224]

In [ ]:
matched_catalog.groupby('cluster_id').get_group(10)['halo_id'].unique()

matched_catalog.groupby('cluster_id').get_group(10)['nmem_cluster'].unique()[0]

clusters_id = matched_catalog['cluster_id'].unique() #This selects the clusters IDs

matched_catalog.groupby(['cluster_id', 'halo_id']).get_group((10, 1162500021323))['nmem_cluster'] #.count()

In [ ]:
matched_catalog.groupby('cluster_id').get_group(10)['shared_mem']

In [ ]:
# shared_frac_cluster = shared_num / cl_group['nmem_cluster'].unique()[0]

In [ ]:
# match_info[match_info['cluster_id'] == 10  ]

cg = matched_catalog.groupby('cluster_id').get_group(10)

cg[cg['shared_frac_cluster'] == cg['shared_frac_cluster'].max()]


In [ ]:
inicio = time.time()
fim = time.time()

print(f'total time = {fim - inicio :.2f} seconds')


In [ ]:
# cluster_groups = match_dataframe_organized.groupby('cluster_id')

# df_test = pd.DataFrame()

# for cl in clusters_id:

#     cluster_group = cluster_groups.get_group(cl)
#     members = list(cluster_group['id'])   
     
#     cluster_group['members'] = cluster_group['coluna'] = [members] * len(cluster_group) 
    
#     pd.concat([df_test, cluster_group], ignore_index=True)


In [ ]:
matched_catalog.groupby('cluster_id').get_group(24)[matched_catalog.groupby('cluster_id').get_group(24)['is_central'] == True]

In [ ]:
# np.mean(matched_catalog.groupby('cluster_id').get_group(24)[['redshift']])

# np.std(matched_catalog.groupby('cluster_id').get_group(10)[['redshift']])

In [ ]:
# clusters_id = matched_catalog['cluster_id'].unique() 

# for cl in clusters_id:

#     z_mean = np.mean(matched_catalog.groupby('cluster_id').get_group(cl)[['redshift']])
#     matched_catalog.groupby('cluster_id').get_group(24)[['redshift', 'cluster_redshift']]



In [ ]:
group = matched_catalog.groupby('cluster_id').get_group(18)
group.groupby('halo_id').get_group(549500032315)['is_central']


In [ ]:
# group[group['shared_frac_cluster'] == group['shared_frac_cluster'].max()]['shared_frac_cluster'].iloc[0]

In [ ]:
# set(best_candidates['halo_id']) - set(bestc['halo_id'])

# set(bestc['halo_id']).symmetric_difference(set(best_candidates['halo_id']))

In [ ]:
# best_candidates[best_candidates['halo_id'] == 632400183300]
best_candidates[best_candidates['cluster_id'] == 27079]

In [ ]:
match_info[match_info['cluster_id'] == 27079]

In [ ]:
cluster_df[cluster_df['cluster_id'] == 10]['cluster_ra'].iloc[0]

halo_central_galaxies[halo_central_galaxies['halo_id'] == 3919900022224]

# best_candidates[best_candidates['halo_id'] == 3919900022224]

In [ ]:
gal11003719599 = truthdf[truthdf['halo_id'] == 3696200176219]
ra = gal11003719599['ra'].values
dec = gal11003719599['dec'].values

In [ ]:
gal11003719599

In [ ]:
cluster1736 = cluster_df[cluster_df['cluster_id'] == 1736][['cluster_id','cluster_ra', 'cluster_dec', ]]

cluster1736['distance'] = angular_distance(cluster1736['cluster_ra'].values, cluster1736['cluster_dec'].values, ra, dec)
cluster1736
# np.sort(list(memberdf[memberdf['cluster_id'] == 1736]['id']))

In [ ]:
# inicio = time.time()

# mt_dataframe2 = mtrmdc2(member_data, cluster_data, truth_data)

# fim = time.time()
# tempo_total = fim - inicio

# print(f"Tempo de execução: {tempo_total/60:.3}min")

In [ ]:
# mt_dataframe

In [ ]:
# mt_table = Table.from_pandas(match_dataframe)

In [ ]:
# mt_table.write('match_ID.fits')

In [ ]:
# match_dataframe_organized = match_dataframe_organized[match_dataframe_organized['cluster_id'] == 23]

In [ ]:
# match_dataframe_organized = match_dataframe_organized[match_dataframe_organized['richness'] > 50]
# m_selection = match_dataframe_organized[np.logical_and(np.log(match_dataframe_organized['richness']) > 5.0,
#                                                        np.log(match_dataframe_organized['richness'] < 5.5))]

# m_selection = m_selection[np.logical_and(m_selection['redshift'] > 0.2, m_selection['redshift'] < 0.4 )]

# m_selection

In [ ]:
# match_dataframe[ match_dataframe['cluster_id'] == 1059]

In [ ]:
# num_mem1 = memberdf['cluster_id'].value_counts()
# cluster_df = pd.merge(pd.DataFrame(num_mem1), cluster_df, how='inner', on=['cluster_id']) 


In [ ]:
num_mem2 = truthdf['halo_id'].value_counts()
truthdf = pd.merge(pd.DataFrame(num_mem2), truthdf, how='inner', on=['halo_id']) 

In [ ]:
truthdf = truthdf.rename(columns={'count': 'nmem_halo'})
cluster_df = cluster_df.rename(columns={'count': 'nmem_cluster'})

In [ ]:
cluster_catalog = pd.merge(memberdf, cluster_df, how='inner', on=['cluster_id'])
# cluster_catalog

In [ ]:
matched_catalog = pd.merge(cluster_catalog, truthdf, how='inner', on=['id'])
matched_catalog

In [ ]:
shared_count = matched_catalog.groupby(['cluster_id', 'halo_id'])['halo_id'].transform('count') 

frac_catalog1 = shared_count / matched_catalog['nmem_cluster']
frac_catalog_halos = shared_count / matched_catalog['nmem_halo']

matched_catalog['shared_mem'] = shared_count
matched_catalog['shared_frac_cluster'] = frac_catalog1
matched_catalog['shared_frac_halo'] = frac_catalog_halos


In [ ]:
matched_catalog[matched_catalog['halo_id'].isin([1544700047300, 1316400047300])]

In [ ]:
    

# def maximum_shared_fraction(cluster_group):
    
#     return cluster_group[cluster_group['shared_frac_cluster'] == cluster_group['shared_frac_cluster'].max()]


# def minimum_distance(cluster_group, max_frac_candidates):

#     candidates_info = cluster_group[cluster_group['halo_id'].isin(max_frac_candidates)]

#     ch_distances = []
        
#     for mfc in max_frac_candidates:

#         candidate = candidates_info[candidates_info['halo_id'] == mfc]
            
#         if True in candidate['is_central'].tolist():

#                candidate_central = candidate[candidate['is_central'] == True]

#                cluster_halo_distance = angular_distance(candidate_central['cluster_ra'].values, candidate_central['cluster_dec'].values,
#                                                                candidate_central['ra'].values, candidate_central['dec'].values)
                
#         else:

#             candidate_ra = np.mean(candidate['ra'])
#             candidate_dec = np.mean(candidate['dec'])
            
#             cluster_halo_distance = angular_distance(candidate['cluster_ra'].values, candidate['cluster_dec'].values,
#                                                                candidate_ra, candidate_dec)
                
            
#         ch_distances.append((mfc, cl, cluster_halo_distance[0]))

        
#     ch_distances_df = pd.DataFrame(ch_distances, columns= ['halo_id', 'cluster_id', 'distance'])
    
#     min_distance_candidates = ch_distances_df[ch_distances_df['distance'] == ch_distances_df['distance'].min()]

#     return min_distance_candidates
    
